In [ ]:
pip install tabulate


In [1]:
#@markdown #1. Install the dependencies 
#dependency conflict 
#DEFINING THE FUNCTIONS

from tabulate import tabulate
from astropy.visualization import simple_norm

from ipywidgets import interact

import numpy as np
import skimage.io
import skimage.filters
import matplotlib.pyplot as plt
import os, random
import shutil 
import zipfile
from tifffile import imread, imsave


import pandas as pd


#Structural similarity (SSIM) index for measuring image quality #can probably change the sigma
def ssim(img1, img2):
  return structural_similarity(img1,img2,data_range=1.,full=True, gaussian_weights=True, use_sample_covariance=False, sigma=1.5)


def normalize(x, pmin=3, pmax=99.8, axis=None, clip=False, eps=1e-20, dtype=np.float32):
    """This function is adapted from Martin Weigert"""
    """Percentile-based image normalization."""

    mi = np.percentile(x,pmin,axis=axis,keepdims=True)
    ma = np.percentile(x,pmax,axis=axis,keepdims=True)
    return normalize_mi_ma(x, mi, ma, clip=clip, eps=eps, dtype=dtype)


def normalize_mi_ma(x, mi, ma, clip=False, eps=1e-20, dtype=np.float32):#dtype=np.float32
    """This function is adapted from Martin Weigert"""
    if dtype is not None:
        x   = x.astype(dtype,copy=False)
        mi  = dtype(mi) if np.isscalar(mi) else mi.astype(dtype,copy=False)
        ma  = dtype(ma) if np.isscalar(ma) else ma.astype(dtype,copy=False)
        eps = dtype(eps)

    try:
        import numexpr
        x = numexpr.evaluate("(x - mi) / ( ma - mi + eps )")
    except ImportError:
        x =                   (x - mi) / ( ma - mi + eps )

    if clip:
        x = np.clip(x,0,1)

    return x

def norm_minmse(gt, x, normalize_gt=True):
    """This function is adapted from Martin Weigert"""

    """
    normalizes and affinely scales an image pair such that the MSE is minimized  
     
    Parameters
    ----------
    gt: ndarray
        the ground truth image      
    x: ndarray
        the image that will be affinely scaled 
    normalize_gt: bool
        set to True of gt image should be normalized (default)
    Returns
    -------
    gt_scaled, x_scaled 
    """
    if normalize_gt:
        gt = normalize(gt, 0.1, 99.9, clip=False).astype(np.float32, copy = False)
    x = x.astype(np.float32, copy=False) - np.mean(x)    
    gt = gt.astype(np.float32, copy=False) - np.mean(gt)    
    scale = np.cov(x.flatten(), gt.flatten())[0, 1] / np.var(x.flatten())
    return gt, scale * x

In [4]:
#@markdown #3. Choose the folders that contain the data to analyse and run to load the data. 
#The source folder must contain both the unprocessed and processed file, or run it twice on both folders
#Here running on the processed files
#separate output folder

Source_folder = "/Users/secchim/Downloads/CellProfiler/movie_pipeline_test/4_drift_corrected" #@param{type:"string"}
# Source_folder = "/Users/secchim/Downloads/CellProfiler/movies2_apply" #@param{type:"string"}
Result_folder = "/Users/secchim/Downloads/CellProfiler/movie_pipeline_test/drift_assessment" #@param{type:"string"}

Analysis_type = "Z-slice" #@param ["Max_projection", "Z-slice"]

Reference_Frame = "Previous" #@param ["First", "Previous"]


#@markdown ##If not Max_projection, choose the Z plane to analyse
Z_plane =  5#@param {type:"number"}

# -------------------------------- Load the stack --------------------------------

# random_choice=random.choice(os.listdir(Source_folder))
for i in os.listdir(Source_folder):
  if i.endswith('Ch2.tif') or i.endswith('xyzCorrected.tif'):
    stack = imread(Source_folder+"/"+i)

    print(stack.shape)

    if Reference_Frame == "Previous":           
      print('The Previous frame will be used as a reference')

    if Reference_Frame == "First": 
      print('The First frame will be used as a reference')


    # perform the max projection

    if Analysis_type == "Max_projection":
      #make max projection
      maxproj = np.max(stack[:,:,:,:],axis = 1)
      print('---------------------------')
      print('max projection shape', maxproj.shape)


    if Analysis_type == "Z-slice":
      maxproj = stack[:,Z_plane,:,:]

      print('---------------------------')
      print('Stack shape:', maxproj.shape)



    # #Display one image

    # f=plt.figure(figsize=(16,8))
    # plt.subplot(1,2,1)
    # plt.imshow(maxproj[0], norm=simple_norm(maxproj[0], percent = 99), interpolation='nearest')

    # plt.axis('off')
    # plt.title('First frame');
    # plt.subplot(1,2,2)
    # plt.imshow(maxproj[1], norm=simple_norm(maxproj[1], percent = 99), interpolation='nearest')
    # plt.axis('off')
    # plt.title('Second frame');



(32, 11, 584, 588)
The Previous frame will be used as a reference
---------------------------
Stack shape: (32, 584, 588)
(21, 12, 506, 505)
The Previous frame will be used as a reference
---------------------------
Stack shape: (21, 506, 505)
(32, 11, 584, 585)
The Previous frame will be used as a reference
---------------------------
Stack shape: (32, 584, 585)
(25, 14, 509, 510)
The Previous frame will be used as a reference
---------------------------
Stack shape: (25, 509, 510)
(25, 14, 510, 505)
The Previous frame will be used as a reference
---------------------------
Stack shape: (25, 510, 505)
(22, 16, 508, 507)
The Previous frame will be used as a reference
---------------------------
Stack shape: (22, 508, 507)
(25, 16, 508, 507)
The Previous frame will be used as a reference
---------------------------
Stack shape: (25, 508, 507)
(21, 12, 508, 506)
The Previous frame will be used as a reference
---------------------------
Stack shape: (21, 508, 506)
(22, 14, 507, 508)
The P

In [5]:
#@markdown #4. Process the data

import csv
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio as psnr

if Analysis_type == "Max_projection":
  Z_plane = ""

for i in os.listdir(Source_folder):
  if not os.path.isdir(os.path.join(Source_folder,i)):
    #if i.endswith('Ch2.tif') or i.endswith('xyzCorrected.tif'): # removed xyz_corrected in the scenario where the raw and corrected files are in different folders
    if i.endswith('Ch2.tif'):
      print('Running QC on: '+i)
      if i.startswith('VWF') and i.endswith('.tif'): #added that line because one hidden file was not tif in the folder
        stack = imread(Source_folder+"/"+i)

        if Analysis_type == "Max_projection":
      
          maxproj = np.max(stack[:,:,:,:],axis = 1)

        if Analysis_type == "Z-slice":
          maxproj = stack[:,Z_plane,:,:]

    # Open and create the csv file that will contain all the QC metrics
        with open(Result_folder+"/"+"QC_metrics_"+i+"_"+Analysis_type+str(Z_plane)+"_"+Reference_Frame+".csv", "w", newline='') as file:
            writer = csv.writer(file)

        # Write the header in the csv file
            writer.writerow(["image #","Z plane","Z plane + 1", "mSSIM", "NRMSE", "PSNR", "Pearson coefficient"])  

            # Initialize the lists
            Z_plane_list = []
            ssim_score_list = []
            Pearson_correlation_coefficient_list = []
              
        # Let's loop through the provided dataset in the QC folders

            for z in range(maxproj.shape[0]-1):

              Z_plane_list.append(z)            
          # -------------------------------- Load the data --------------------------------
              if Reference_Frame == "Previous":           
                test_GT = maxproj[z+1]                     
                test_source = maxproj[z]

              if Reference_Frame == "First":           
                test_GT = maxproj[z+1]                     
                test_source = maxproj[0]

          # Normalize the images wrt each other by minimizing the MSE between GT and Source image
              test_GT_norm,test_source_norm = norm_minmse(test_GT, test_source, normalize_gt=True)

          # -------------------------------- Calculate the metric maps and save them --------------------------------

          # Calculate the SSIM maps #structural similarity index
              index_SSIM_GTvsSource, img_SSIM_GTvsSource = ssim(test_GT_norm, test_source_norm)

              ssim_score_list.append(index_SSIM_GTvsSource)

          #Save ssim_maps

                #img_SSIM_GTvsSource_8bit = (img_SSIM_GTvsSource* 255).astype("uint8")
                #io.imsave(QC_model_path+'/'+QC_model_name+"/Quality Control/"+str(checkpoints)+"/SSIM_GTvsSource_"+shortname_no_PNG+'.tif',img_SSIM_GTvsSource_8bit)
          
          # Calculate the Root Squared Error (RSE) maps
              img_RSE_GTvsSource = np.sqrt(np.square(test_GT_norm - test_source_norm))

          # Save SE maps
                #img_RSE_GTvsSource_8bit = (img_RSE_GTvsSource* 255).astype("uint8")
                #io.imsave(QC_model_path+'/'+QC_model_name+"/Quality Control/"+str(checkpoints)+"/RSE_GTvsSource_"+shortname_no_PNG+'.tif',img_RSE_GTvsSource_8bit)


          # -------------------------------- Calculate the RSE metrics and save them --------------------------------

          # Normalised Root Mean Squared Error (here it's valid to take the mean of the image)
              NRMSE_GTvsSource = np.sqrt(np.mean(img_RSE_GTvsSource))
            
          # We can also measure the peak signal to noise ratio between the images
              PSNR_GTvsSource = psnr(test_GT_norm,test_source_norm,data_range=1.0)


              cm1 = np.corrcoef(test_GT_norm.flat, test_source_norm.flat) #outputs a flat number
              r1 = cm1[0, 1]
              Pearson_correlation_coefficient_list.append(r1)

              writer.writerow([i,str(z),str(z+1),str(index_SSIM_GTvsSource),str(NRMSE_GTvsSource),str(PSNR_GTvsSource),str(r1)])

  # All data is now processed saved




Running QC on: VWF_062_MS220421_M2_movie_P4_Ch2_xyzCorrected.tif
Running QC on: VWF_053_MS220209_M1_movie2homeostasis_P18kuo_Ch3_xyzCorrected.tif
Running QC on: VWF_062_MS220421_M2_movie_P6_Ch4_xyzCorrected.tif
Running QC on: VWF_049_MS220201_M1_Snap_movie5depletion_P14touchMK_Ch2_xyzCorrected.tif
Running QC on: VWF_049_MS220201_M1_Snap_movie5depletion_P10spot_Ch4_xyzCorrected.tif
Running QC on: VWF_048_MS220118_M3_movie4plt_P12+_Ch4_xyzCorrected.tif
Running QC on: VWF_049_MS220201_M1_Snap_movie5depletion_P152kuo_Ch1_xyzCorrected.tif
Running QC on: VWF_053_MS220209_M1_movie2homeostasis_P24kuomk_Ch2_xyzCorrected.tif
Running QC on: VWF_048_MS220118_M3_movie4plt_P27+_Ch3_xyzCorrected.tif
Running QC on: VWF_049_MS220201_M1_Snap_movie5depletion_P9kuo2_Ch2_xyzCorrected.tif
Running QC on: VWF_048_MS220118_M3_movie4plt_P6DP+_Ch2_xyzCorrected.tif
Running QC on: VWF_053_MS220209_M1_movie2homeostasis_P16DP_Ch2_xyzCorrected.tif
Running QC on: VWF_062_MS220421_M2_movie_P7_Ch1_xyzCorrected.tif
Runni

In [6]:
#@markdown #5. Choose the folders that contain the data to analyse and run to load the data. 
#The source folder must contain both the unprocessed and processed file, or run it twice on both folders
#Solution here running on the raw data
#separate output folder

Source_folder = "/Users/secchim/Downloads/CellProfiler/movie_pipeline_test/2_channel_split_tif" #@param{type:"string"}
# Source_folder = "/Users/secchim/Downloads/CellProfiler/movies2_apply" #@param{type:"string"}
Result_folder = "/Users/secchim/Downloads/CellProfiler/movie_pipeline_test/drift_assessment" #@param{type:"string"}

Analysis_type = "Z-slice" #@param ["Max_projection", "Z-slice"]

Reference_Frame = "Previous" #@param ["First", "Previous"]


#@markdown ##If not Max_projection, choose the Z plane to analyse
Z_plane =  5#@param {type:"number"}

# -------------------------------- Load the stack --------------------------------

# random_choice=random.choice(os.listdir(Source_folder))
for i in os.listdir(Source_folder):
  if i.endswith('Ch2.tif') or i.endswith('xyzCorrected.tif'):
    stack = imread(Source_folder+"/"+i)

    print(stack.shape)

    if Reference_Frame == "Previous":           
      print('The Previous frame will be used as a reference')

    if Reference_Frame == "First": 
      print('The First frame will be used as a reference')


    # perform the max projection

    if Analysis_type == "Max_projection":
      #make max projection
      maxproj = np.max(stack[:,:,:,:],axis = 1)
      print('---------------------------')
      print('max projection shape', maxproj.shape)


    if Analysis_type == "Z-slice":
      maxproj = stack[:,Z_plane,:,:]

      print('---------------------------')
      print('Stack shape:', maxproj.shape)



    # #Display one image

    # f=plt.figure(figsize=(16,8))
    # plt.subplot(1,2,1)
    # plt.imshow(maxproj[0], norm=simple_norm(maxproj[0], percent = 99), interpolation='nearest')

    # plt.axis('off')
    # plt.title('First frame');
    # plt.subplot(1,2,2)
    # plt.imshow(maxproj[1], norm=simple_norm(maxproj[1], percent = 99), interpolation='nearest')
    # plt.axis('off')
    # plt.title('Second frame');

(22, 12, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (22, 512, 512)
(32, 9, 590, 590)
The Previous frame will be used as a reference
---------------------------
Stack shape: (32, 590, 590)
(22, 12, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (22, 512, 512)
(21, 10, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (21, 512, 512)
(32, 9, 590, 590)
The Previous frame will be used as a reference
---------------------------
Stack shape: (32, 590, 590)
(32, 9, 590, 590)
The Previous frame will be used as a reference
---------------------------
Stack shape: (32, 590, 590)
(21, 10, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (21, 512, 512)
(21, 10, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (21, 512, 512)
(21, 10, 512, 512)
The Prev

In [7]:
#@markdown #6. Process the raw data

import csv
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio as psnr

if Analysis_type == "Max_projection":
  Z_plane = ""

for i in os.listdir(Source_folder):
  if not os.path.isdir(os.path.join(Source_folder,i)):
    if i.endswith('Ch2.tif') or i.endswith('xyzCorrected.tif'):
      print('Running QC on: '+i)
      if i.startswith('VWF') and i.endswith('.tif'): #added that line because one hidden file was not tif in the folder
        stack = imread(Source_folder+"/"+i)

        if Analysis_type == "Max_projection":
      
          maxproj = np.max(stack[:,:,:,:],axis = 1)

        if Analysis_type == "Z-slice":
          maxproj = stack[:,Z_plane,:,:]

    # Open and create the csv file that will contain all the QC metrics
        with open(Result_folder+"/"+"QC_metrics_"+i+"_"+Analysis_type+str(Z_plane)+"_"+Reference_Frame+".csv", "w", newline='') as file:
            writer = csv.writer(file)

        # Write the header in the csv file
            writer.writerow(["image #","Z plane","Z plane + 1", "mSSIM", "NRMSE", "PSNR", "Pearson coefficient"])  

            # Initialize the lists
            Z_plane_list = []
            ssim_score_list = []
            Pearson_correlation_coefficient_list = []
              
        # Let's loop through the provided dataset in the QC folders

            for z in range(maxproj.shape[0]-1):

              Z_plane_list.append(z)            
          # -------------------------------- Load the data --------------------------------
              if Reference_Frame == "Previous":           
                test_GT = maxproj[z+1]                     
                test_source = maxproj[z]

              if Reference_Frame == "First":           
                test_GT = maxproj[z+1]                     
                test_source = maxproj[0]

          # Normalize the images wrt each other by minimizing the MSE between GT and Source image
              test_GT_norm,test_source_norm = norm_minmse(test_GT, test_source, normalize_gt=True)

          # -------------------------------- Calculate the metric maps and save them --------------------------------

          # Calculate the SSIM maps #structural similarity index
              index_SSIM_GTvsSource, img_SSIM_GTvsSource = ssim(test_GT_norm, test_source_norm)

              ssim_score_list.append(index_SSIM_GTvsSource)

          #Save ssim_maps

                #img_SSIM_GTvsSource_8bit = (img_SSIM_GTvsSource* 255).astype("uint8")
                #io.imsave(QC_model_path+'/'+QC_model_name+"/Quality Control/"+str(checkpoints)+"/SSIM_GTvsSource_"+shortname_no_PNG+'.tif',img_SSIM_GTvsSource_8bit)
          
          # Calculate the Root Squared Error (RSE) maps
              img_RSE_GTvsSource = np.sqrt(np.square(test_GT_norm - test_source_norm))

          # Save SE maps
                #img_RSE_GTvsSource_8bit = (img_RSE_GTvsSource* 255).astype("uint8")
                #io.imsave(QC_model_path+'/'+QC_model_name+"/Quality Control/"+str(checkpoints)+"/RSE_GTvsSource_"+shortname_no_PNG+'.tif',img_RSE_GTvsSource_8bit)


          # -------------------------------- Calculate the RSE metrics and save them --------------------------------

          # Normalised Root Mean Squared Error (here it's valid to take the mean of the image)
              NRMSE_GTvsSource = np.sqrt(np.mean(img_RSE_GTvsSource))
            
          # We can also measure the peak signal to noise ratio between the images
              PSNR_GTvsSource = psnr(test_GT_norm,test_source_norm,data_range=1.0)


              cm1 = np.corrcoef(test_GT_norm.flat, test_source_norm.flat) #outputs a flat number
              r1 = cm1[0, 1]
              Pearson_correlation_coefficient_list.append(r1)

              writer.writerow([i,str(z),str(z+1),str(index_SSIM_GTvsSource),str(NRMSE_GTvsSource),str(PSNR_GTvsSource),str(r1)])

  # All data is now processed saved

Running QC on: VWF_048_MS220118_M3_movie4plt_P27+_Ch2.tif
Running QC on: VWF_062_MS220421_M2_movie_P4_Ch2.tif
Running QC on: VWF_048_MS220118_M3_movie4plt_P12+_Ch2.tif
Running QC on: VWF_053_MS220209_M1_movie2homeostasis_P24kuomk_Ch2.tif
Running QC on: VWF_062_MS220421_M2_movie_P7_Ch2.tif
Running QC on: VWF_062_MS220421_M2_movie_P6_Ch2.tif
Running QC on: VWF_053_MS220209_M1_movie2homeostasis_P14partlygreen_Ch2.tif
Running QC on: VWF_053_MS220209_M1_movie2homeostasis_P16DP_Ch2.tif
Running QC on: VWF_053_MS220209_M1_movie2homeostasis_P18kuo_Ch2.tif
Running QC on: VWF_048_MS220118_M3_movie4plt_P6DP+_Ch2.tif
Running QC on: VWF_053_MS220209_M1_movie2homeostasis_P26kuovascu_Ch2.tif
Running QC on: VWF_049_MS220201_M1_Snap_movie5depletion_P14touchMK_Ch2.tif
Running QC on: VWF_048_MS220118_M3_movie4plt_P22+_Ch2.tif
Running QC on: VWF_049_MS220201_M1_Snap_movie5depletion_P152kuo_Ch2.tif
Running QC on: VWF_049_MS220201_M1_Snap_movie5depletion_P9kuo2_Ch2.tif
Running QC on: VWF_049_MS220201_M1_Snap

In [8]:
#@markdown #. Showing and SAVING all 
##MATCHING UNCORRECTED AND FAST4DREG CORRECTED FILES

Result_folder = "/Users/secchim/Downloads/CellProfiler/movie_pipeline_test/drift_assessment" #@param{type:"string"}
QCResult_folder='/Users/secchim/Downloads/CellProfiler/movie_pipeline_test/drift_assessment_png'

def show_QC_results(file1): #file1, file2, requires a list as an argument
  # files= os.listdir(Result_folder)#
  # file1=[i for i in os.listdir(Result_folder) if "Corrected" not in i] #if file already has corrected don't look at them , runs quicker than forloop, called list comprehension
  df1 = pd.read_csv (Result_folder+"/"+file1)
  ind=file1.index(".tif")
  rep=file1[ind:]
  file2=file1.replace(rep, '_xyzCorrected'+rep)
  # file3=file1.replace(rep, 'drift'+rep)
  df2 = pd.read_csv (Result_folder+"/"+file2)
  # df3 = pd.read_csv (Result_folder+"/"+file3)
  df1.set_index("image #", inplace=True)
  df2.set_index("image #", inplace=True)
  # df3.set_index("image #", inplace=True)
  # print(tabulate((df1,df2), headers='keys', tablefmt='psql'))

  Z_plane_list1 = df1['Z plane + 1'].values.tolist()
  Z_plane_list2 = df2['Z plane + 1'].values.tolist()
  # Z_plane_list3 = df3['Z plane + 1'].values.tolist()
  ssim_score_list1 = df1['mSSIM'].values.tolist()
  ssim_score_list2 = df2['mSSIM'].values.tolist()
  # ssim_score_list3 = df3['mSSIM'].values.tolist()
  Pearson_correlation_coefficient_list1 = df1['Pearson coefficient'].values.tolist()
  Pearson_correlation_coefficient_list2 = df2['Pearson coefficient'].values.tolist()
  # Pearson_correlation_coefficient_list3 = df3['Pearson coefficient'].values.tolist()



# -------------------------------- Display --------------------------------

  plt.figure(figsize=(20,5))
  plt.plot(Z_plane_list1, ssim_score_list1, label="SSIM_raw")
  plt.plot(Z_plane_list2, ssim_score_list2, label="SSIM_Fast4Dcorrected")
  # plt.plot(Z_plane_list3, ssim_score_list3, label="SSIM_ImageJcorrected")
  plt.title('Timepoints vs. SSIM')
  plt.ylabel('SSIM')
  plt.xlabel('Timepoints')
  plt.legend()
  # plt.savefig(full_QC_model_path+'/Quality Control/SSIMvsCheckpoint_data.png',bbox_inches='tight',pad_inches=0)
  plt.savefig(QCResult_folder+("/"+file1+'SSIMvsCheckpoint_data.png'),bbox_inches='tight',pad_inches=0)
  #plt.show()
  plt.close()

  plt.figure(figsize=(20,5))
  plt.plot(Z_plane_list1, Pearson_correlation_coefficient_list1, label="Pearson coefficient_raw")
  plt.plot(Z_plane_list2, Pearson_correlation_coefficient_list2, label="Pearson coefficient_Fast4Dcorrected")
  # plt.plot(Z_plane_list3, Pearson_correlation_coefficient_list3, label="Pearson coefficient_ImageJcorrected")
  plt.title('Timepoints vs. Pearson coefficient')
  plt.ylabel('Pearson coefficient')
  plt.xlabel('Timepoints')
  plt.legend()
#plt.savefig(full_QC_model_path+'/Quality Control/lpipsvsCheckpoint_data.png',bbox_inches='tight',pad_inches=0)
  plt.savefig(QCResult_folder+("/"+file1+'lpipsvsCheckpoint_data.png'),bbox_inches='tight',pad_inches=0)
 # plt.show()
  plt.close()



for i in os.listdir(Result_folder):
  if (".tif" in i and "Corrected" not in i and ".DS_Store" not in i): 
    # print(i)
    show_QC_results(i)
    print(i)

#vasculature P12 is the problem
    

QC_metrics_VWF_062_MS220421_M2_movie_P7_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_048_MS220118_M3_movie4plt_P22+_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_053_MS220209_M1_movie2homeostasis_P14partlygreen_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_053_MS220209_M1_movie2homeostasis_P26kuovascu_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_062_MS220421_M2_movie_P6_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_048_MS220118_M3_movie4plt_P12+_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_049_MS220201_M1_Snap_movie5depletion_P152kuo_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_053_MS220209_M1_movie2homeostasis_P16DP_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_049_MS220201_M1_Snap_movie5depletion_P14touchMK_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_053_MS220209_M1_movie2homeostasis_P18kuo_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_049_MS220201_M1_Snap_movie5depletion_P12DP_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_048_MS220118_M3_movie4plt_P6DP+_Ch2.tif_Z-slice5_Previous.csv
QC_m

In [11]:
Result_folder = "/Users/secchim/Downloads/CellProfiler/QC2" #@param{type:"string"}
QCResult_folder='/Users/secchim/Downloads/CellProfiler/QC2png'

print(os.listdir(Result_folder))


['QC_metrics_VWF_045_MS211208_M2_movie7_P27_Ch2_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie7_P25_Ch4_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie6_P23_Ch4_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie7_P27_Ch1_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie3_P21_Ch4_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie8_P26_Ch4_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie7_P25_Ch1_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie6_P23_Ch1_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie8_P26_Ch2_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie3_P21_Ch2_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie8_P26_Ch1_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2

In [8]:
#This cell will return a list of movies that should be checked because pearson correlation is low


for file1 in os.listdir(Result_folder):
  if ".tif" in file1 and "Corrected" not in file1: 
    ind=file1.index(".tif")
    rep=file1[ind:]
    file2=file1.replace(rep, '_xyzCorrected'+rep)
    df2 = pd.read_csv (Result_folder+"/"+file2)

    Pearson_correlation_coefficient_list2 = df2['Pearson coefficient'].values.tolist()
        
    for t in Pearson_correlation_coefficient_list2 :
        if t<0.4:
            print(file2)
            break